In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [5]:
NHL = pd.read_csv('InjStatus.csv')
NHL.head()
# NHL.columns

,Unnamed: 0,POSITION,HEIGHT,WEIGHT,PLAYER,AGE,Injury Status
0,0,LW,"6'2""",214,Abdelkader Justin,32,Injured
1,1,LW,"5'11""",196,Aberg Pontus,25,Injured
2,2,RW,"5'9""",171,Abramov Vitaly,21,Not Injured
3,3,C,"5'10""",205,Acciari Noel,27,Not Injured
4,4,LW,"6'0""",199,Agostino Kenny,27,Not Injured


In [7]:
data = NHL.values
X = data[:, 1:6]
y = data[:, 6]

In [8]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

In [9]:
for label, original_class in zip(encoded_y, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)

Original Class: Injured
Encoded Label: 0
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encode

Encoded Label: 0
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Not Injured
Encoded Label: 1
------------
Orig

Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Not Injured
Encoded Label: 1
------------
Original Class: Injured
Encoded Label: 0
------------
Original Class: Injured
Encoded Label: 0
-

In [10]:
from keras.utils import to_categorical

# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
one_hot_y

Using TensorFlow backend.


array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [ ]:
OneHotEncode